# Anti Spoofing CNN

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, accuracy_score

## Data Download

In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("faber24/lcc-fasd")

print("Path to dataset files:", path)

Path to dataset files: /home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1


In [18]:
import os

for entry in os.scandir(path):
    if entry.is_dir:
        for entry2 in os.scandir(entry.path):
            print(entry2.path)
    

/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_training
/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_evaluation
/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_development


In [19]:
data_splits_paths = {
    "training":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_training",
    "evaluation":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_evaluation",
    "development":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_development"
}


## Creating Dataloader

In [20]:
data = datasets.ImageFolder(data_splits_paths["training"])
data.class_to_idx

{'real': 0, 'spoof': 1}

In [21]:
transform = transforms.Compose([
    transforms.Resize(256),  # Resize images to 224x224
    transforms.CenterCrop(224),
    transforms.ToTensor(),          # Convert images to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Assuming you have a directory with 'train' and 'test' folders and each folder has class subfolders
train_dataset = datasets.ImageFolder(root=data_splits_paths['training'], transform=transform)
test_dataset = datasets.ImageFolder(root=data_splits_paths['evaluation'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Training

In [22]:
def simple_train_model(model,criterion,optimizer,epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):  # loop over the dataset multiple times
        model.train(True)
        running_loss = 0.0
        running_f1 = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)

            optimizer.zero_grad()  # zero the parameter gradients
            
            outputs = model(inputs)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            running_loss += loss
            optimizer.step()
        
            y_pred, y_true = np.round(outputs.detach().cpu().numpy(), decimals=0), labels.detach().cpu().numpy()
            running_f1 += f1_score(y_pred=y_pred, y_true=y_true)
            running_acc += accuracy_score(y_pred=y_pred, y_true=y_true)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/i:.4f}, F1 Score: {running_f1/i:.4f}, Accuracy: {running_acc/i:.4f}')



In [23]:
def train_model(model,criterion,optimizer,epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):  # loop over the dataset multiple times
        model.train(True)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)

            optimizer.zero_grad()  # zero the parameter gradients
            
            outputs = model(inputs)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            y_pred, y_true = np.round(outputs.detach().cpu().numpy(), decimals=0), labels.detach().cpu().numpy()
            f1 = f1_score(y_pred=y_pred, y_true=y_true)
            acc = accuracy_score(y_pred=y_pred, y_true=y_true)
            print(f'\tbatch {i}, Loss: {loss:.4f}, F1 Score: {f1:.4f}, Accuracy: {acc:.4f}')


        running_vloss = 0.0
        running_f1 = 0.0
        running_acc = 0.0
        model.eval()
        with torch.no_grad():
            for e, vdata in enumerate(test_loader):
                vinputs, vlabels = vdata
                vinputs, vlabels = vinputs.to(device), vlabels.to(device).to(torch.float32)
                voutputs = model(vinputs)
                voutputs = voutputs.squeeze(1)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss
                y_pred, y_true = np.round(voutputs.detach().cpu().numpy(), decimals=0), vlabels.detach().cpu().numpy()
                running_f1 += f1_score(y_pred=y_pred, y_true=y_true)
                running_acc += accuracy_score(y_pred=y_pred, y_true=y_true)
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_vloss/e:.4f}, F1 Score: {running_f1/e:.4f}, Accuracy: {running_acc/e:.4f}')


In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [30]:
class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [36]:
class EvenDeeperCNN(nn.Module):
    def __init__(self):
        super(EvenDeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

In [25]:
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 14.7456, F1 Score: 0.9228, Accuracy: 0.8561
Epoch 2/5, Loss: 14.8023, F1 Score: 0.9228, Accuracy: 0.8558
Epoch 3/5, Loss: 14.8254, F1 Score: 0.9224, Accuracy: 0.8556
Epoch 4/5, Loss: 14.8023, F1 Score: 0.9228, Accuracy: 0.8558
Epoch 5/5, Loss: 14.7793, F1 Score: 0.9230, Accuracy: 0.8561


In [31]:
model = DeeperCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 0.3152, F1 Score: 0.9306, Accuracy: 0.8821
Epoch 2/5, Loss: 0.1827, F1 Score: 0.9650, Accuracy: 0.9374
Epoch 3/5, Loss: 0.1058, F1 Score: 0.9810, Accuracy: 0.9653
Epoch 4/5, Loss: 0.0618, F1 Score: 0.9914, Accuracy: 0.9831
Epoch 5/5, Loss: 0.0548, F1 Score: 0.9933, Accuracy: 0.9862


In [37]:
model = EvenDeeperCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 0.3472, F1 Score: 0.9255, Accuracy: 0.8627
Epoch 2/5, Loss: 0.2279, F1 Score: 0.9529, Accuracy: 0.9156
Epoch 3/5, Loss: 0.1403, F1 Score: 0.9733, Accuracy: 0.9517
Epoch 4/5, Loss: 0.0908, F1 Score: 0.9856, Accuracy: 0.9730
Epoch 5/5, Loss: 0.0643, F1 Score: 0.9906, Accuracy: 0.9817
